In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import optuna

# 加载和预处理数据
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)


# 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()  # 28*28二维图像展平成一维向量，size=784
        self.fc1 = nn.Linear(28 * 28, 30)  # 784维映射到30维
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # 舍去50%的神经元
        self.fc2 = nn.Linear(30, 10)  # 30维映射到10维

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


def add_noise(inputs, noise_level):
    noise = torch.randn_like(inputs) * noise_level
    return inputs + noise


def add_weight_noise(params, noise_level):
    with torch.no_grad():
        for param in params:
            noise = torch.randn_like(param) * noise_level
            param.add_(noise)


In [2]:

def objective(trial):
    input_noise_level = trial.suggest_float("input_noise_level", 0.0, 0.2)  # 优化输入噪声水平
    weight_noise_level = trial.suggest_float("weight_noise_level", 0.0, 0.01)  # 优化权重噪声水平

    # 实例化模型、定义损失函数和优化器
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    num_epochs = 10  # 可以根据需要调整
    for epoch in range(num_epochs):
        model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, input_noise_level)  # 添加输入噪声
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            add_weight_noise(model.parameters(), weight_noise_level)  # 添加权重噪声
            optimizer.step()
            running_loss += loss.item()

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, input_noise_level)  # 添加输入噪声
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


# 运行优化
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 获取最佳噪声水平
best_input_noise_level = study.best_params["input_noise_level"]
best_weight_noise_level = study.best_params["weight_noise_level"]
print("Best input noise level:", best_input_noise_level)
print("Best weight noise level:", best_weight_noise_level)


In [3]:

# 使用最佳噪声水平重新训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # 将模型设置为训练模式，以便在前向传播时启用 Dropout 和 BatchNorm
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = add_noise(inputs, best_input_noise_level)  # 使用最佳输入噪声水平添加噪声
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        add_weight_noise(model.parameters(), best_weight_noise_level)  # 使用最佳权重噪声水平添加噪声
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    # 在测试集上验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 在验证过程中禁用梯度计算，以提高计算速度和节省内存。
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = add_noise(inputs, best_input_noise_level)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')
